In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load dữ liệu

In [1]:
import pandas as pd

folder = r'C:\Users\The Anh\OneDrive\Máy tính\Project\v4_0.42\\'

In [5]:
df_text_train = pd.read_csv(folder+'features_text_in_img_train_v2.csv')
df_text_img_train = pd.read_csv(folder+'features_text_train_v2.csv')
df_img_train = pd.read_csv(folder+'image_features.csv')

In [62]:
df_text_img_test= pd.read_csv(folder+'features_text_prtest.csv')
df_text_test = pd.read_csv(folder+'features_text_in_img_prtest.csv')
df_img_test = pd.read_csv(folder+'features_img_prtest.csv')

In [6]:
df_train = pd.read_excel(r"C:\Users\The Anh\OneDrive\Máy tính\Project\Data\file_text_in_image.xlsx")

In [3]:
import ast

def string_to_tensor_array(string_data):
    # Chuyển đổi chuỗi thành mảng NumPy
    array_data = ast.literal_eval(string_data)
    # Chuyển đổi mảng NumPy thành tensor với kiểu dữ liệu float32
    return array_data

In [7]:
df_img_train['features'] = df_img_train['features'].apply(string_to_tensor_array)
df_img_test['features'] = df_img_test['features'].apply(string_to_tensor_array)

In [18]:
df_img_test['features'] = df_img_test['features'].apply(string_to_tensor_array)
df_text_test['features'] = df_text_test['features'].apply(string_to_tensor_array)
df_text_img_test['features'] = df_text_img_test['features'].apply(string_to_tensor_array)

In [8]:
df_text_train['features'] = df_text_train['features'].apply(string_to_tensor_array)
df_text_img_train['features'] = df_text_img_train['features'].apply(string_to_tensor_array)

df_text_test['features'] = df_text_test['features'].apply(string_to_tensor_array)
df_text_img_test['features'] = df_text_img_test['features'].apply(string_to_tensor_array)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import f1_score
import random
from torch.utils.data import Dataset, DataLoader

# Tạo dataset

In [7]:
class SarcasmDataset(Dataset):
    def __init__(self, df):
        self.text_feats = df['text_feats'].tolist()
        self.img_feats = df['img_feats'].tolist()
        self.ocr_feats = df['ocr_feats'].tolist()
        self.labels = df['label'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text_feats = torch.tensor(self.text_feats[idx], dtype=torch.float32)
        img_feats = torch.tensor(self.img_feats[idx], dtype=torch.float32)
        ocr_feats = torch.tensor(self.ocr_feats[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return {
            'text_feats': text_feats,
            'img_feats': img_feats,
            'ocr_feats': ocr_feats,
            'label': label
        }

In [9]:
import json
import pandas as pd

def load_json_to_dataframe(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return pd.DataFrame(data).transpose()

# File paths
train_path = r"C:\Users\The Anh\OneDrive\Máy tính\Project\Data\vimmsd-train.json"

# Load data into DataFrames
d = load_json_to_dataframe(train_path)

In [10]:
df = pd.DataFrame({'text_feats':df_text_train['features'].tolist(),'img_feats':df_img_train['features'].tolist(),'ocr_feats':df_text_img_train['features'].tolist(),'label':d['label'].tolist()})

In [11]:
label_map = {'text-sarcasm':0,'image-sarcasm': 1, 'multi-sarcasm':2, 'not-sarcasm':3}
df['label']=df['label'].map(label_map)

In [12]:
df.head()

,text_feats,img_feats,ocr_feats,label
0,"[-0.0707225501537323, 0.6053339838981628, -0.3...","[-0.3173622190952301, 0.21320968866348267, 0.1...","[-0.01944945938885212, 0.2595173716545105, -0....",2
1,"[0.01768472231924534, 0.17569275200366974, -0....","[0.16594907641410828, -0.09616951644420624, -0...","[0.0899106115102768, 0.3913036584854126, -0.05...",3
2,"[-0.0707225501537323, 0.6053339838981628, -0.3...","[-0.6251385807991028, -0.16424448788166046, 0....","[0.002900111023336649, 0.07815682142972946, -0...",2
3,"[0.05841079726815224, 0.10343050211668015, -0....","[-0.2023581564426422, 0.20617759227752686, 0.2...","[0.05580971762537956, 0.17241831123828888, 0.0...",3
4,"[-0.0707225501537323, 0.6053339838981628, -0.3...","[-0.6315056085586548, 0.3270147442817688, -0.1...","[-0.0949886366724968, 0.05473310127854347, -0....",2


In [ ]:
df.groupby('label').describe()

In [14]:
df_0 = df[df['label']==0]
df_1 = df[df['label']==1]
df_2 = df[df['label']==2]
df_3 = df[df['label']==3]

In [15]:
from sklearn.model_selection import train_test_split

In [35]:
def creat_df (df_0,df_1,df_2,df_3):
  df_temp = pd.concat([df_0,df_1.sample(len(df_0), replace=False),df_2.sample(len(df_0), replace=False),df_3.sample(len(df_0), replace=False)])
  train, val = train_test_split(df_temp,test_size=0.2,stratify=df_temp['label'])
  dataset = SarcasmDataset(train)
  val_tr = SarcasmDataset(val)

  train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  val_loader = DataLoader(val_tr, batch_size=16, shuffle=False)
  return train_loader, val_loader

# Mô hình

In [24]:
class SarcasmDetector(nn.Module):
    def __init__(self, text_dim, img_dim, ocr_dim, hidden_dim=128):  # Giảm hidden_dim
        super(SarcasmDetector, self).__init__()

        # Main text encoder
        self.text_encoder = nn.Sequential(
            nn.Linear(text_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.5),  # Tăng tỷ lệ dropout
            nn.LayerNorm(hidden_dim)
        )

        # Image feature encoder
        self.img_encoder = nn.Sequential(
            nn.Linear(img_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.LayerNorm(hidden_dim)
        )

        # OCR text encoder
        self.ocr_encoder = nn.Sequential(
            nn.Linear(ocr_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.LayerNorm(hidden_dim)
        )

        # Cross attention
        self.text_img_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=8, dropout=0.5)
        self.text_ocr_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=8, dropout=0.5)

        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.LayerNorm(hidden_dim)
        )

        # Task-specific heads
        self.text_only_classifier = nn.Linear(hidden_dim, 4)
        self.img_only_classifier = nn.Linear(hidden_dim * 2, 4)
        self.combined_classifier = nn.Linear(hidden_dim, 4)

    def forward(self, text_feats, img_feats, ocr_feats):
      # Encode individual modalities
      text_encoded = self.text_encoder(text_feats)  # [batch_size, hidden_dim]
      img_encoded = self.img_encoder(img_feats)      # [batch_size, hidden_dim]
      ocr_encoded = self.ocr_encoder(ocr_feats)      # [batch_size, hidden_dim]

      # Text-image attention
      text_img_attn, _ = self.text_img_attention(
          text_encoded.unsqueeze(0),  # [1, batch_size, hidden_dim]
          img_encoded.unsqueeze(0),    # [1, batch_size, hidden_dim]
          img_encoded.unsqueeze(0)     # [1, batch_size, hidden_dim]
      )

      # Text-OCR attention
      text_ocr_attn, _ = self.text_ocr_attention(
          text_encoded.unsqueeze(0),   # [1, batch_size, hidden_dim]
          ocr_encoded.unsqueeze(0),     # [1, batch_size, hidden_dim]
          ocr_encoded.unsqueeze(0)      # [1, batch_size, hidden_dim]
      )

      # Combine features for final prediction
      combined = self.fusion(torch.cat([
          text_img_attn.squeeze(0),  # [batch_size, hidden_dim]
          text_ocr_attn.squeeze(0),  # [batch_size, hidden_dim]
          text_encoded                 # [batch_size, hidden_dim]
      ], dim=-1))  # [batch_size, hidden_dim * 3]

      # Final output
      final_output = self.combined_classifier(combined)  # [batch_size, 4] hoặc số lớp cần dự đoán

      return final_output

In [18]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import autocast, GradScaler

In [9]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score

def calculate_class_weights(labels):
    """ Tính toán trọng số cho các lớp dựa trên số lượng mẫu trong từng lớp """
    class_counts = np.bincount(labels)
    total_count = len(labels)
    class_weights = total_count / (len(class_counts) * class_counts)
    return torch.FloatTensor(class_weights)

def weighted_focal_loss(outputs, targets, weights, gamma=2.0):
    """ Tính toán Focal Loss với trọng số """
    ce_loss = F.cross_entropy(outputs, targets, reduction='none', weight=weights)
    pt = torch.exp(-ce_loss)
    focal_loss = ((1 - pt) ** gamma * ce_loss).mean()
    return focal_loss

def evaluate_task(predictions, targets):
    """ Đánh giá F1 macro và accuracy cho một task cụ thể """
    if len(predictions) == 0 or len(targets) == 0:
        return 0.0, 0.0

    f1 = f1_score(targets, predictions, average='macro')
    accuracy = accuracy_score(targets, predictions)
    return f1, accuracy

def evaluate(model, val_loader, class_weights):
    model.eval()
    predictions = []
    targets = []
    total_loss = 0
    num_batches = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Không theo dõi gradient trong chế độ đánh giá
        for batch in val_loader:
            text_feats = batch['text_feats']
            img_feats = batch['img_feats']
            ocr_feats = batch['ocr_feats']
            batch_targets = batch['label']

            # Forward pass
            outputs = model(text_feats, img_feats, ocr_feats)  # Nhận đầu ra chỉ từ một classifier

            # Tính loss
            loss = weighted_focal_loss(outputs, batch_targets, class_weights)
            total_loss += loss.item()

            # Lưu predictions và targets
            preds = outputs.argmax(dim=1)
            predictions.extend(preds.cpu().numpy())
            targets.extend(batch_targets.cpu().numpy())

            # Tính đúng/sai cho accuracy
            total_correct += (preds == batch_targets).sum().item()
            total_samples += batch_targets.size(0)

            num_batches += 1

    # Tính F1 macro và accuracy
    overall_f1 = f1_score(targets, predictions, average='macro')
    accuracy = total_correct / total_samples if total_samples > 0 else 0.0
    avg_loss = total_loss / num_batches if num_batches > 0 else float('inf')

    return {
        'loss': avg_loss,
        'f1_macro': overall_f1,
        'accuracy': accuracy
    }


In [20]:
import numpy as np

class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        """
        Khởi tạo Early Stopping.

        :param patience: Số epoch tối đa cho phép mà không có cải thiện
        :param delta: Sự cải thiện tối thiểu cần thiết để tính là cải thiện
        """
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, score):
        """
        Gọi hàm để kiểm tra cải thiện.

        :param score: F1 score hiện tại
        """
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


In [25]:
# Khởi tạo mô hình với các chiều đặc trưng của text, image, và OCR
text_dim = 768  # ví dụ chiều của đặc trưng văn bản sau BERT
img_dim = 768  # Kích thước đặc trưng ảnh sau ViT
ocr_dim = 768  # Ví dụ chiều của đặc trưng OCR, giả định giống với ảnh
hidden_dim = 256  # Kích thước ẩn

# Khởi tạo mô hình
model = SarcasmDetector(
    text_dim=text_dim,
    img_dim=img_dim,
    ocr_dim=ocr_dim,
    hidden_dim=hidden_dim
)

# Kiểm tra cấu trúc của mô hình
print(model)


SarcasmDetector(
  (text_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (img_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (ocr_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (text_img_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (text_ocr_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (fu

# Train

In [22]:
class_weights=torch.tensor([1.0]*4)

In [23]:
import os
def train_model(model, optimizer, num_epochs, class_weights):
    model.train()  # Đặt mô hình vào chế độ huấn luyện
    total_loss = 0.0

    for epoch in range(num_epochs):
        count = 0
        train_loader, val_loader = creat_df (df_0,df_1,df_2,df_3)
        for batch in train_loader:
            optimizer.zero_grad()  # Đặt gradient về 0

            # Lấy các đầu vào từ batch
            text_feats = batch['text_feats']
            img_feats = batch['img_feats']
            ocr_feats = batch['ocr_feats']
            labels = batch['label']  # Không còn task_type nữa

            # Forward pass
            outputs = model(text_feats, img_feats, ocr_feats)

            # Tính loss
            loss = weighted_focal_loss(outputs, labels, class_weights)
            loss.backward()  # Lan truyền gradient
            optimizer.step()  # Cập nhật weights

            total_loss += loss.item()

        # Tính trung bình loss cho mỗi epoch
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

        # Đánh giá mô hình trên tập validation
        eval_results = evaluate(model, val_loader, class_weights)
        print(f'Validation Loss: {eval_results["loss"]:.4f}, F1 macro: {eval_results["f1_macro"]:.4f}, Accuracy: {eval_results["accuracy"]}')

    # Lưu mô hình sau khi huấn luyện xong
    model_save_path = os.path.join(folder, 'model_2.pth')  # Đường dẫn để lưu mô hình
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved to {model_save_path}')

    return avg_loss

# Train

In [ ]:
# Khởi tạo optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

# Số lượng epoch
num_epochs = 250  # Số lượng epoch bạn muốn huấn luyện
# Gọi hàm train với đầy đủ tham số
F1 = train_model(
    model=model,
    optimizer=optimizer,
    num_epochs=num_epochs,
    class_weights=class_weights)


Epoch [1/50], Average Loss: 0.6168
Validation Loss: 0.1247, F1 macro: 0.8326, Accuracy: 0.8387096774193549
Epoch [2/50], Average Loss: 0.7583
Validation Loss: 0.1545, F1 macro: 0.9042, Accuracy: 0.9032258064516129
Epoch [3/50], Average Loss: 0.9165
Validation Loss: 0.1251, F1 macro: 0.9039, Accuracy: 0.9032258064516129
Epoch [4/50], Average Loss: 1.0578
Validation Loss: 0.1454, F1 macro: 0.8517, Accuracy: 0.8548387096774194
Epoch [5/50], Average Loss: 1.2356
Validation Loss: 0.1528, F1 macro: 0.9024, Accuracy: 0.9032258064516129
Epoch [6/50], Average Loss: 1.3849
Validation Loss: 0.2054, F1 macro: 0.7697, Accuracy: 0.7903225806451613
Epoch [7/50], Average Loss: 1.5393
Validation Loss: 0.1943, F1 macro: 0.7670, Accuracy: 0.7741935483870968
Epoch [8/50], Average Loss: 1.6830
Validation Loss: 0.1467, F1 macro: 0.8363, Accuracy: 0.8387096774193549
Epoch [9/50], Average Loss: 1.8272
Validation Loss: 0.1291, F1 macro: 0.8842, Accuracy: 0.8870967741935484
Epoch [10/50], Average Loss: 1.9846
V

# Predict

In [34]:
test_pth = r"C:\Users\The Anh\OneDrive\Máy tính\Project\Data\vimmsd-public-test.json"
t = load_json_to_dataframe(test_pth)

In [63]:
# File paths
test_pth = r"C:\Users\The Anh\OneDrive\Máy tính\Project\Data\vimmsd-private-test.json"

# Load data into DataFrames
t = load_json_to_dataframe(test_pth)

In [64]:
df_test = pd.DataFrame({'text_feats':df_text_test['features'].tolist(),'img_feats':df_img_test['features'].tolist(),'ocr_feats':df_text_img_test['features'].tolist(),'label':t['label'].tolist()})

In [51]:
label_map = {'text-sarcasm':0,'image-sarcasm': 1, 'multi-sarcasm':2, 'not-sarcasm':3}
df_test['label']=df_test['label'].map(label_map)

In [52]:
df_test

,text_feats,img_feats,ocr_feats,label
0,"[-0.010315310209989548, 0.20558097958564758, -...","[-0.4676307439804077, 0.4149308502674103, 0.11...","[0.22576826810836792, 0.7014867067337036, -0.1...",NaN
1,"[-0.010315310209989548, 0.20558097958564758, -...","[0.2279101014137268, -0.11422616243362427, 0.0...","[0.0378059446811676, 0.3451228141784668, -0.05...",NaN
2,"[-0.010315310209989548, 0.20558097958564758, -...","[0.16580697894096375, 0.03786355257034302, 0.3...","[0.09640201181173325, 0.37234261631965637, -0....",NaN
3,"[-0.010315310209989548, 0.20558097958564758, -...","[0.19905760884284973, 0.04752121865749359, 0.0...","[0.09665238857269287, 0.3772967755794525, -0.0...",NaN
4,"[0.11253988742828369, 0.54109787940979, -0.297...","[-0.042777374386787415, 0.017328515648841858, ...","[-0.0087127685546875, 0.2464548647403717, -0.0...",NaN
...,...,...,...,...
1499,"[0.04230928048491478, 0.15914270281791687, -0....","[0.07648631185293198, 0.6541545391082764, 0.13...","[-0.01727587729692459, 0.06590449064970016, -0...",NaN
1500,"[0.09558481723070145, 0.16480980813503265, -0....","[-0.5109578967094421, 0.29594194889068604, 0.3...","[0.02531014196574688, 0.17944417893886566, -0....",NaN
1501,"[-0.005629384890198708, 0.20771880447864532, -...","[-0.6135715246200562, 0.06028905510902405, 0.1...","[-0.06860345602035522, 0.13520707190036774, -0...",NaN
1502,"[-0.11225829273462296, 0.3060818016529083, -0....","[0.19864866137504578, 0.6988522410392761, 0.23...","[0.008414838463068008, 0.19441823661327362, -0...",NaN


In [65]:
import torch
import ast

# Apply ast.literal_eval if your columns are stored as strings
df_test['img_feats'] = df_test['img_feats'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_test['text_feats'] = df_test['text_feats'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_test['ocr_feats'] = df_test['ocr_feats'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convert to tensors
text_feats = torch.tensor(df_test['text_feats'].values.tolist(), dtype=torch.float32)
img_feats = torch.tensor(df_test['img_feats'].values.tolist(), dtype=torch.float32)
ocr_feats = torch.tensor(df_test['ocr_feats'].values.tolist(), dtype=torch.float32)

# Now you can create the TensorDataset and DataLoader



In [66]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(text_feats, img_feats, ocr_feats)
unlabeled_loader = DataLoader(dataset, batch_size=32, shuffle=False)  # Bạn có thể điều chỉnh batch_size

In [67]:
import os

# Corrected code to join paths
model_path = os.path.join(folder, 'model_1.pth')
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode


C:\Users\The Anh\AppData\Local\Temp\ipykernel_32604\294811166.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


SarcasmDetector(
  (text_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (img_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (ocr_encoder): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (text_img_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (text_ocr_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (fu

In [81]:
def predict_with_index(model):
    model.eval()  # Đặt mô hình vào chế độ đánh giá
    predictions = []
    with torch.no_grad():  # Không theo dõi gradient
        for batch in unlabeled_loader:
            text_feats, img_feats, ocr_feats = batch

            # Forward pass
            outputs = model(text_feats, img_feats, ocr_feats)  # Lấy đầu ra từ mô hình

            # Lấy nhãn dự đoán
            preds = outputs.argmax(dim=1).cpu().numpy()  # Dự đoán nhãn với đầu ra có độ chính xác cao nhất

            # Lưu kết hợp với chỉ số
            for i in range(len(preds)):
                predictions.append((i, preds[i]))  # Lưu index và nhãn
    return predictions

# Dự đoán trên tập dữ liệu chưa gán nhãn
predictions_with_index = predict_with_index(model)

# Chuyển đổi kết quả về DataFrame
predictions_df = pd.DataFrame(predictions_with_index, columns=['index', 'predicted_labels'])

# Gán lại chỉ số cho DataFrame
predictions_df.set_index('index', inplace=True)

predictions_df



,predicted_labels
index,
0,3
1,3
2,3
3,3
4,3
...,...
27,2
28,3
29,2


In [82]:
predictions_df['id'] = predictions_df.index  # Thêm cột id chứa giá trị index


In [83]:
label_map = {'text-sarcasm':0,'image-sarcasm': 1, 'multi-sarcasm':2, 'not-sarcasm':3}


In [84]:
reverse_label_map = {v: k for k, v in label_map.items()}


In [85]:
predictions_df['predicted_labels'] = predictions_df['predicted_labels'].map(reverse_label_map)

In [86]:
predictions_df.groupby('predicted_labels').count()

,id
predicted_labels,
image-sarcasm,74
multi-sarcasm,899
not-sarcasm,530
text-sarcasm,1


In [88]:
predictions_df.to_csv('Las.csv')